In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.conf import SparkConf
config = SparkConf()
# config.set("property", "value")
config.setMaster("local").setAppName("FileFormats")

from pyspark.sql import SparkSession
# spark Session, entry point for Spark SQL, DataFrame
spark = SparkSession.builder\
                    .config(conf=config)\
                    .getOrCreate()

sc = spark.sparkContext

22/05/19 00:04:00 WARN Utils: Your hostname, ubuntu-virtual-machine resolves to a loopback address: 127.0.1.1; using 192.168.174.129 instead (on interface ens33)
22/05/19 00:04:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/05/19 00:04:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/19 00:04:05 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/05/19 00:04:05 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/05/19 00:04:05 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [3]:
# inferSchema will cause performance issue while loading

movieDf = spark.read.format("csv")\
                .option("header", True)\
                .option("inferSchema", True)\
                .load("hdfs://localhost:9000/movies")

In [4]:
movieDf.rdd.getNumPartitions()

1

In [5]:
ratingDf = spark.read.format("csv")\
                .option("header", True)\
                .option("inferSchema", True)\
                .load("hdfs://localhost:9000/ratings")

In [6]:
# convert movies to json 
movieDf.write.mode("overwrite")\
                .json("hdfs://localhost:9000/movies-json")

In [7]:
# convert ratings to json
ratingDf.write.mode("overwrite")\
                .json("hdfs://localhost:9000/ratings-json")

In [8]:
# READ JSON FILE into DataFrame
movieJsonDf = spark.read.format("json")\
                .option("inferSchema", True)\
                .load("hdfs://localhost:9000/movies-json")

movieJsonDf.printSchema()
movieJsonDf.show(5)

root
 |-- genres: string (nullable = true)
 |-- movieId: long (nullable = true)
 |-- title: string (nullable = true)

+--------------------+-------+--------------------+
|              genres|movieId|               title|
+--------------------+-------+--------------------+
|Adventure|Animati...|      1|    Toy Story (1995)|
|Adventure|Childre...|      2|      Jumanji (1995)|
|      Comedy|Romance|      3|Grumpier Old Men ...|
|Comedy|Drama|Romance|      4|Waiting to Exhale...|
|              Comedy|      5|Father of the Bri...|
+--------------------+-------+--------------------+
only showing top 5 rows



In [9]:
# READ JSON FILE into DataFrame
ratingJsonDf = spark.read.format("json")\
                .option("inferSchema", True)\
                .load("hdfs://localhost:9000/ratings-json")

ratingJsonDf.printSchema()
ratingJsonDf.show(5)

root
 |-- movieId: long (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- userId: long (nullable = true)

+-------+------+---------+------+
|movieId|rating|timestamp|userId|
+-------+------+---------+------+
|      1|   4.0|964982703|     1|
|      3|   4.0|964981247|     1|
|      6|   4.0|964982224|     1|
|     47|   5.0|964983815|     1|
|     50|   5.0|964982931|     1|
+-------+------+---------+------+
only showing top 5 rows



In [10]:
# parquet
# Write data frame into parquet format

movieDf.write.mode("overwrite")\
                .parquet("hdfs://localhost:9000/movies-parquet")

ratingDf.write.mode("overwrite")\
                .parquet("hdfs://localhost:9000/ratings-parquet")

In [11]:
# READ Parquet FILE into DataFrame
# inferSchema not needed as parquet has schema itself
movieParquetDf = spark.read.format("parquet")\
                .load("hdfs://localhost:9000/movies-parquet")

movieParquetDf.printSchema()
movieParquetDf.show(5)

root
 |-- movieId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
+-------+--------------------+--------------------+
only showing top 5 rows



In [12]:
# READ Parquet FILE into DataFrame
# inferSchema not needed as parquet has schema itself
ratingParquetDf = spark.read.format("parquet")\
                .load("hdfs://localhost:9000/ratings-parquet")

ratingParquetDf.printSchema()
ratingParquetDf.show(5)

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
+------+-------+------+---------+
only showing top 5 rows

